# GLM Demo: multivariate noise stimulus

This is a demo notebook for the GLM estimation algorithm. We simulate data using the GLM, and then try to estimate parameters with the GLM. This notebook demos the use of a multivariate stimulus like a spectrogram, although the stimulus used here is more like 1D visual white noise.

In [ ]:
from __future__ import print_function, division
import os
import sys
import imp
import numpy as np

import mat_neuron._model as mat
from dstrf import strf, mle

# plotting packages
%matplotlib inline
import matplotlib.pyplot as plt # plotting functions
import seaborn as sns           # data visualization package
sns.set_style("whitegrid")

In [ ]:
# model parameters: (ω, α1, α2, τ1, τ2, tref)
matparams = np.asarray([7, 100, 2, 10, 200, 2], dtype='d')
model_dt = 0.5

# data parameters
duration = 400000
n_samples = int(duration / model_dt)
n_assim = 1
n_test = 5

In [ ]:
# STRF: keep this very simple for proof of principle
stim_dt = 10.0
kscale = 10
nfreq = 14
ntau  = 30
ntbas = 8

# raised-cosine basis functions
kcosbas = strf.cosbasis(ntau, ntbas)

from scipy.signal import resample
filts = np.load('../../filters.npz')
k1 = resample(filts['bbs'] * kscale, nfreq, axis=0)[:,ntau-1::-1]

plt.imshow(k1, cmap='jet', aspect='auto')

In [ ]:
# generate some random data to fit
np.random.seed(1)
mat.random_seed(1)
stim_dt = 10.0
upsample = int(stim_dt / model_dt)
stim = np.random.randn(nfreq, int(n_samples / (stim_dt / model_dt)))
stim[:,:100] = 0
plt.imshow(stim, aspect='auto')

In [ ]:
def predict_spikes(V, params, dt, upsample):
    omega, a1, a2, t1, t2, tref = params
    return mat.predict_poisson(V - omega, (a1, a2), (t1, t2), tref, 
                               dt, upsample)

data = []
V = strf.convolve(stim, k1)
for i in range(n_assim + n_test):
    spikes = predict_spikes(V, matparams, model_dt, upsample)
    H = mat.adaptation(spikes, matparams[3:5], model_dt)
    z = np.nonzero(spikes)[0]
    d = {"H": H,
         "duration": duration,
         "spike_t": z, 
         "spike_v": spikes,
        }
    data.append(d)

# split into assimilation and test sets
assim_data = data[:n_assim]
test_data = data[n_assim:]

In [ ]:
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)
ax1.plot(V)
for i, d in enumerate(data):
    ax2.vlines(d["spike_t"], i, i + 0.5)
ax1.set_xlim(0, 8000 // upsample)
ax2.set_xlim(0, 8000);

## Estimate parameters

First we'll estimate the full RF (using the raised cosine basis set to compress time)

In [ ]:
# initial guess of parameters using ML
ntbas = 8
kcosbas = strf.cosbasis(ntau, ntbas)
spike_v = np.stack([d["spike_v"] for d in assim_data], axis=1)
spike_h = np.stack([d["H"] for d in assim_data], axis=2)
try:
    mlest = mle.mat(stim, kcosbas, spike_v, spike_h, stim_dt, model_dt, nlin="exp")
except TypeError:
    mlest = mle.mat(stim, kcosbas, spike_v, spike_h, stim_dt, model_dt, nlin="exp")
%time w0 = mlest.estimate()

In [ ]:
print("True rate and adaptation parameters:", matparams[:3])
print("MLE rate and adaptation parameters:", w0[:3])
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, figsize=(6, 6))

k1c = strf.to_basis(k1, kcosbas)
rf_sta = strf.as_matrix(mlest.sta(), kcosbas)
rf_mle = strf.as_matrix(w0[3:], kcosbas)
axes[0, 0].imshow(k1, cmap='jet', aspect='auto')
axes[0, 0].set_title("True RF")
axes[0, 1].imshow(strf.from_basis(k1c, kcosbas), cmap='jet', aspect='auto')
axes[0, 1].set_title("RF from cosine basis")
axes[1, 0].imshow(rf_sta, cmap='jet', aspect='auto')
axes[1, 0].set_title("STA")
axes[1, 1].imshow(rf_mle, cmap='jet', aspect='auto')
axes[1, 1].set_title("MLE")

We can also reduce parameter counts by factorizing the STRF (as in Thorson et al). This doesn't make much of a difference with white noise stimuli but it can really help when there are strong correlations.

In [ ]:
krank = 1
mlest = mle.matfact(stim, kcosbas, krank, spike_v, spike_h, stim_dt, model_dt, nlin="exp")
%time w0 = mlest.estimate()

In [ ]:
print("True rate and adaptation parameters:", matparams[:3])
print("MLE rate and adaptation parameters:", w0[:3])
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, figsize=(6, 6))

k1c = strf.to_basis(k1, kcosbas)
rf_sta = strf.as_matrix(mlest.sta(), kcosbas)
rf_mle = strf.from_basis(strf.defactorize(w0[3:], nfreq, krank), kcosbas)
axes[0, 0].imshow(k1, cmap='jet', aspect='auto')
axes[0, 0].set_title("True RF")
axes[0, 1].imshow(strf.from_basis(k1c, kcosbas), cmap='jet', aspect='auto')
axes[0, 1].set_title("RF from cosine basis")
axes[1, 0].imshow(rf_sta, cmap='jet', aspect='auto')
axes[1, 0].set_title("STA")
axes[1, 1].imshow(rf_mle, cmap='jet', aspect='auto')
axes[1, 1].set_title("MLE (rank-{})".format(krank));